In [1]:
import pyopencl as cl
import numpy as np

import os
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'
os.environ['PYOPENCL_CTX'] = '1'

(n, m, p) = (3, 4, 5)

# a = np.random.randn(n, m).astype(np.float32)
# b = np.random.randn(m, p).astype(np.float32)
a = np.random.randint(2, size=(n*m))
b = np.random.randint(2, size=(m*p))
c = np.zeros((n*p), dtype=np.float32)

a = a.astype(np.float32)
b = b.astype(np.float32)

In [3]:
ctx = cl.Context(dev_type=cl.device_type.ALL)
queue = cl.CommandQueue(ctx)


In [4]:
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a)
b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
c_buf = cl.Buffer(ctx, mf.WRITE_ONLY, c.nbytes)

In [5]:
prg = cl.Program(ctx, """
    __kernel void multiply(ushort n,
    ushort m, ushort p, __global float *a,
    __global float *b, __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = 0.0f;
      int rowC = gid/p;
      int colC = gid%p;
      __global float *pA = &a[rowC*m];
      __global float *pB = &b[colC];
      for(int k=0; k<m; k++)
      {
         pB = &b[colC+k*p];
         c[gid] += (*(pA++))*(*pB);
      }
    }
    """).build()

In [6]:
prg.multiply(queue, c.shape, None,
             np.uint16(n), np.uint16(m), np.uint16(p),
             a_buf, b_buf, c_buf)

a_mul_b = np.empty_like(c)
cl.enqueue_copy(queue, a_mul_b, c_buf)

In [7]:
print(a.reshape(n, m))

[[0. 1. 1. 0.]
 [1. 1. 0. 0.]
 [1. 0. 1. 1.]]


In [8]:
print(b.reshape(m, p))

[[0. 1. 1. 0. 0.]
 [1. 1. 1. 1. 0.]
 [0. 1. 1. 0. 1.]
 [0. 1. 1. 0. 0.]]


In [9]:
print(a_mul_b.reshape(n, p))

[[1. 2. 2. 1. 1.]
 [1. 2. 2. 1. 0.]
 [0. 3. 3. 0. 1.]]
